In [1]:
import os
import json
import re
import shutil
input_folder = "encoded_yosys"
output_folder = "done_all"
import torch
import torch.nn.functional as F


In [2]:
# Use list comprehension to create a list of file paths
verilog_files = [os.path.join(input_folder, filename) for filename in os.listdir(input_folder) if filename.endswith('.txt')]

# Print the list of file paths
print(len(verilog_files))
print(type(verilog_files[0]))
print(verilog_files[0])


396
<class 'str'>
encoded_yosys\adder10_synth.txt


In [3]:
ground_truth_labels = [0,1,2,3,4,5,6,7,8,9,10,11,12,13, 14, 15] 
labels_torch = torch.tensor(ground_truth_labels)

# One-hot encode the labels
# one_hot_labels = F.one_hot(labels_torch, num_classes=16)
# one_hot_labels = F.one_hot(labels_torch, num_classes=9)
# one_hot_labels = F.one_hot(labels_torch, num_classes=15)
one_hot_labels = F.one_hot(labels_torch, num_classes=16)

print(one_hot_labels[0])

tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])


In [4]:

labels_dict = dict()
def label_verilog_file(file_name):
    # label_mapping = {
    #     'adder': 0, 'ALU': 1, 'comparator': 2, 'decoder': 3,
    #     'encoder': 4, 'mult': 5, 'mux': 6, 'pe': 7, 'sub': 8
    # }
    label_mapping = {
        'adder': 0, 'comparator': 1, 'decoder': 2,
        'encoder': 3, 'mult': 4, 'mux': 5, 'pe': 6, 'sub': 7, 'and': 8, 'or': 9, 'not': 10, 'nand': 11, 'nor': 12, 'xnor': 13,'xor':14, 'seg': 15
    }
    
    pattern = r"([a-zA-Z]+)(\d+)?"
    match = re.match(pattern, file_name)
    if match:
        base_name = match.group(1)
        if base_name in label_mapping:
            labels_dict[base_name] = labels_dict.get(base_name, 0) + 1
            return one_hot_labels[label_mapping[base_name]].tolist()
        
    return None


def add_label_to_verilog_file(input_file_path, output_folder):
    if input_file_path.endswith('.txt'):
        with open(input_file_path, "r") as file:
            loaded_data = json.load(file)
            label = label_verilog_file(os.path.basename(input_file_path))
            if label is not None and [label] not in loaded_data:  # Check if label already exists
                loaded_data.append([label])  # Add label as the third list
                output_file_path = os.path.join(output_folder, os.path.basename(input_file_path))
                with open(output_file_path, "w") as output_file:
                    json.dump(loaded_data, output_file)
                return True
    return False



# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Process files in the input folder
for file_name in os.listdir(input_folder):
    file_path = os.path.join(input_folder, file_name)
    if os.path.isfile(file_path):
        if not add_label_to_verilog_file(file_path, output_folder):
            print(f"Failed to label: {file_path}")



In [5]:
failed_files = []

def get_files_in_folder(input_folder):
    file_list = []
    for file_name in os.listdir(input_folder):
        file_path = os.path.join(input_folder, file_name)
        if os.path.isfile(file_path):
            file_list.append(file_path)
    return file_list

# Replace 'input_folder' with the path to the folder you want to read files from
verilog_files = get_files_in_folder(output_folder)
print(len(verilog_files))
print(failed_files)
print(verilog_files)

# for v in verilog_files:
#     with open(v, "r") as file:
#         loaded_data = json.load(file)
#         # print(loaded_data[0])
        
#         for node in loaded_data[0]:
#             if len(node) != 4:
#                 failed_files.append(v)
#                 break
            
#how to save list in a txt file


396
[]
['done_all\\adder10_synth.txt', 'done_all\\adder11_synth.txt', 'done_all\\adder12_synth.txt', 'done_all\\adder13_synth.txt', 'done_all\\adder14_synth.txt', 'done_all\\adder15_synth.txt', 'done_all\\adder16_synth.txt', 'done_all\\adder17_synth.txt', 'done_all\\adder18_synth.txt', 'done_all\\adder19_synth.txt', 'done_all\\adder1_synth.txt', 'done_all\\adder20_synth.txt', 'done_all\\adder21_synth.txt', 'done_all\\adder22_synth.txt', 'done_all\\adder23_synth.txt', 'done_all\\adder24_synth.txt', 'done_all\\adder25_synth.txt', 'done_all\\adder26_synth.txt', 'done_all\\adder27_synth.txt', 'done_all\\adder28_synth.txt', 'done_all\\adder2_synth.txt', 'done_all\\adder3_synth.txt', 'done_all\\adder4_synth.txt', 'done_all\\adder5_synth.txt', 'done_all\\adder6_synth.txt', 'done_all\\adder7_synth.txt', 'done_all\\adder8_synth.txt', 'done_all\\adder9_synth.txt', 'done_all\\and10_gate_synth.txt', 'done_all\\and11_gate_synth.txt', 'done_all\\and12_gate_synth.txt', 'done_all\\and13_synth.txt', 'd

# reduce classes: 
    -Decoder
    -Encoder
    -Mux

removed seg and bcd classes

In [14]:
# print(labels_dict)
# data_str_keys = {str(key): value for key, value in labels_dict.items()}
with open("labels_dict.txt", "w") as file:
    json.dump(labels_dict, file)

labels_dict

{'adder': 28,
 'and': 30,
 'comparator': 22,
 'decoder': 32,
 'encoder': 25,
 'mult': 29,
 'mux': 27,
 'nand': 31,
 'nor': 30,
 'not': 16,
 'or': 27,
 'pe': 21,
 'sub': 11,
 'xnor': 30,
 'xor': 28}

In [15]:
def get_files_in_folder(input_folder):
    file_list = []
    for file_name in os.listdir(input_folder):
        file_path = os.path.join(input_folder, file_name)
        if os.path.isfile(file_path):
            file_list.append(file_path)
    return file_list

In [8]:
def preprocessing(folder_path):
    
    file_list = get_files_in_folder(folder_path)
    
    return file_list

In [ ]:
# classes_dict = dict()
# for file in verilog_files:
#     if file.endswith('.txt'):
#         with open(file, "r") as f:
#             loaded_data = json.load(file)
#             temp = 
#             for sublist in loaded_data[0]:
                